# BKF Crawler

In [6]:
# ==========================================================
#   BKF e-Flora Thailand Crawler (Vol. 2–16)
#   ดึงข้อมูล Accepted Name / Thailand / Distribution / Ecology
#   Author: ChatGPT (GPT-5) | Updated: 2025-10-16
# ==========================================================
# !pip install selenium webdriver-manager pandas beautifulsoup4

import re, time, random, requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ---------- SETTINGS ----------
BASE = "https://botany.dnp.go.th/eflora/"
VOLUMES = range(2, 17)           # Volume 2–16
OUT_CSV = "bkf_eflora_species_vol2_16.csv"
SLEEP_BASE = 1.2                 # พักระหว่าง request
WAIT_SEC = 20                    # Selenium explicit wait (วินาที)
HEADLESS = True                  # ตั้งเป็น False ถ้าอยากดู browser จริง

In [13]:
# ==========================================================
# BKF e-Flora Thailand Crawler (Vol. 2–16, Full + Resume)
# - Family discovery: a[href^='florafamily.html'] (dedup)
# - Family name: from URL ?factsheet=<FAMILY> (robust) + fallback <p class="intro">Family :</p>
# - Click "List of lower taxa" tab before reading Genus
# - Drilldown: Family -> Genus -> Species
# - Extract: Accepted Name / Thailand / Distribution / Ecology
# - Sleep: 1–2s (randomized) to be polite
# - Resume: skip species_url already saved in CSV
# Updated: 2025-10-16
# ==========================================================

import os, re, time, random, requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urlparse, parse_qs

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ---------- SETTINGS ----------
BASE        = "https://botany.dnp.go.th/eflora/"
VOLUMES     = range(2, 3)                 # เลือกช่วงเล่มได้ เช่น range(2,3) เพื่อเทส Vol.2 ก่อน
OUT_CSV     = "bkf_eflora_species_vol2_16.csv"
SLEEP_BASE  = 1.2                          # พักพื้นฐาน (จะถูกสุ่มคูณเล็กน้อย)
WAIT_SEC    = 25                           # Explicit wait ของ Selenium
HEADLESS    = True                         # False ถ้าอยากเห็นเบราว์เซอร์จริง

# ---------- UTILITIES ----------
def nap(mult: float = 1.0):
    """Sleep แบบสุ่ม (ประมาณ 1–2 วินาที)"""
    time.sleep(SLEEP_BASE * mult * (0.85 + random.random()*0.3))

def clean(t: str | None):
    return re.sub(r"\s+", " ", t).strip() if t else None

def to_abs(url: str | None):
    """แปลง relative URL -> absolute URL"""
    if not url:
        return None
    if url.startswith("http"):
        return url
    return requests.compat.urljoin(BASE, url)

def soup_from_driver(driver):
    return BeautifulSoup(driver.page_source, "html.parser")

def find_label_value(soup: BeautifulSoup, label_regex: str):
    """
    หา value ที่ตามหลัง label (Accepted Name / Distribution / Ecology / Thailand) แบบยืดหยุ่น
    """
    tag = soup.find(string=re.compile(label_regex, re.I))
    if tag:
        cur = tag
        for _ in range(12):
            cur = cur.next_element
            if cur is None:
                break
            txt = clean(cur.get_text(" ", strip=True) if hasattr(cur, "get_text") else str(cur))
            if txt and not re.search(label_regex, txt, re.I):
                return re.sub(r"^[:\s]+", "", txt)
    # fallback: grep จาก text ทั้งหน้า
    full = soup.get_text("\n", strip=True)
    m = re.search(rf"{label_regex}\s*:?\s*(.+)", full, re.I)
    return clean(m.group(1)) if m else None

def setup_driver(headless: bool = True):
    """ตั้งค่า Selenium Chrome"""
    opts = webdriver.ChromeOptions()
    opts.add_argument("--window-size=1280,900")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--user-agent=Mozilla/5.0")
    if headless:
        opts.add_argument("--headless=new")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    return driver

# ---------- FAMILY DISCOVERY (robust) ----------
def get_families_for_volume(driver, vol: int):
    """
    เปิดหน้า Volume -> ดึง family จาก a[href^='florafamily.html'] (dedup)
    ตั้งชื่อ family จากพารามิเตอร์ ?factsheet= ใน URL เป็นหลัก
    ถ้าไม่มี ค่อย fallback ไปอ่านจาก <p class="intro"><strong>Family :</strong> ...</p> ภายในหน้า family
    """
    vol_url = f"{BASE}floramainvol.html?vol={vol}"
    driver.get(vol_url)
    WebDriverWait(driver, WAIT_SEC).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(3)  # รอ JS render ตาราง

    soup = soup_from_driver(driver)
    families, seen = [], set()

    for a in soup.select("a[href^='florafamily.html']"):
        href = to_abs(a.get("href", ""))
        if not href or href in seen:
            continue
        seen.add(href)

        # 1) ชื่อจาก ?factsheet=
        fam_name = None
        try:
            qs = parse_qs(urlparse(href).query)
            fam_name = qs.get("factsheet", [None])[0]
        except Exception:
            pass

        # 2) ถ้ายังไม่มีชื่อ ลองเข้าไปอ่านจากหน้า family
        if not fam_name:
            driver.get(href)
            try:
                WebDriverWait(driver, WAIT_SEC).until(EC.presence_of_element_located((By.CSS_SELECTOR, "p.intro")))
            except TimeoutException:
                pass
            s = soup_from_driver(driver)
            p = s.select_one("p.intro")
            if p and "Family" in p.get_text():
                txt = p.get_text(" ", strip=True)
                fam_name = re.sub(r".*Family\s*:\s*", "", txt)

        families.append((clean(fam_name) if fam_name else None, href))

    return families

# ---------- GENUS & SPECIES ----------
def get_genus_links(driver, family_url: str):
    """
    เข้า family แล้ว 'คลิกแท็บ List of lower taxa' ก่อนค่อยดึงลิงก์ Genus
    """
    driver.get(to_abs(family_url))

    # รอหน้าโหลด
    try:
        WebDriverWait(driver, WAIT_SEC).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        time.sleep(2)

    # คลิกแท็บ 'List of lower taxa' (ตัวสะกด/โครงสร้างอาจต่างกันเล็กน้อยในบาง Family)
    clicked = False
    for locator in [
        (By.XPATH, "//a[contains(., 'List of lower taxa')]"),
        (By.XPATH, "//li[contains(., 'List of lower taxa')]//a"),
        (By.CSS_SELECTOR, "a[href*='lower'], a[href*='#lower'], li a[href*='lower']")
    ]:
        try:
            el = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(locator))
            el.click()
            clicked = True
            break
        except Exception:
            pass
    if not clicked:
        time.sleep(1)

    # รอให้ลิงก์ Genus ปรากฏ
    try:
        WebDriverWait(driver, WAIT_SEC).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='floragenus.html']"))
        )
    except TimeoutException:
        time.sleep(2)

    s = soup_from_driver(driver)
    gens = []
    for a in s.select("a[href*='floragenus.html']"):
        gens.append((clean(a.get_text()), to_abs(a.get("href", ""))))

    # สำรองจาก DOM (กัน soup ไม่ครบ)
    if not gens:
        elems = driver.find_elements(By.CSS_SELECTOR, "a[href*='floragenus.html']")
        gens = [(clean(e.text), to_abs(e.get_attribute("href"))) for e in elems]

    # unique
    uniq, seen = [], set()
    for name, url in gens:
        if url and url not in seen:
            seen.add(url)
            uniq.append((name, url))
    return uniq

def get_species_links(driver, genus_url: str):
    driver.get(to_abs(genus_url))
    try:
        WebDriverWait(driver, WAIT_SEC).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='floraspecies.html']"))
        )
    except TimeoutException:
        time.sleep(2)
    s = soup_from_driver(driver)
    sp = [to_abs(a.get("href", "")) for a in s.select("a[href*='floraspecies.html']")]
    if not sp:
        elems = driver.find_elements(By.CSS_SELECTOR, "a[href*='floraspecies.html']")
        sp = [to_abs(e.get_attribute("href")) for e in elems]
    # unique
    out, seen = [], set()
    for u in sp:
        if u and u not in seen:
            seen.add(u)
            out.append(u)
    return out

def parse_species_page(driver, sp_url: str):
    driver.get(to_abs(sp_url))
    try:
        WebDriverWait(driver, WAIT_SEC).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        pass
    s = soup_from_driver(driver)

    accepted     = find_label_value(s, r"Accepted\s*Name")
    thailand     = find_label_value(s, r"Thailand")
    distribution = find_label_value(s, r"Distribution")
    ecology      = find_label_value(s, r"Ecology")

    sci_name = None
    if accepted:
        m = re.match(r"([A-Z][a-zA-Z-]+)\s+([a-z\-]+)", accepted)
        if m:
            sci_name = f"{m.group(1)} {m.group(2)}"

    return {
        "accepted_name": clean(accepted),
        "species_scientific_name": clean(sci_name),
        "thailand": clean(thailand),
        "distribution": clean(distribution),
        "ecology": clean(ecology),
    }

# ---------- MAIN (with Resume) ----------
def crawl_all(volumes=VOLUMES, out_csv=OUT_CSV):
    # Resume: โหลดไฟล์เดิม
    rows, done_species = [], set()
    if os.path.exists(out_csv):
        try:
            old = pd.read_csv(out_csv)
            rows = old.to_dict("records")
            if "species_url" in old.columns:
                done_species = set(old["species_url"].dropna().astype(str).tolist())
            print(f"🧩 Resume mode: โหลด {len(rows)} แถวเดิม, species ที่ทำแล้ว {len(done_species)}")
        except Exception as e:
            print(f"⚠️ โหลดไฟล์เดิมไม่สำเร็จ: {e}")

    driver = setup_driver(headless=HEADLESS)

    try:
        for vol in volumes:
            families = get_families_for_volume(driver, vol)
            print(f"\n=== Volume {vol} ===")
            print(f"  → พบ {len(families)} families (หลังกรองซ้ำ)")

            for fi, (family_name, family_url) in enumerate(families, 1):
                family_name = clean(family_name) or "Unknown Family"
                nap()
                genus_links = get_genus_links(driver, family_url)
                print(f"    [{fi}/{len(families)}] {family_name}: {len(genus_links)} genera")

                for gi, (genus_name, genus_url) in enumerate(genus_links, 1):
                    nap()
                    species_links = get_species_links(driver, genus_url)

                    for si, sp_url in enumerate(species_links, 1):
                        if not sp_url or sp_url in done_species:
                            continue
                        nap(1.1)
                        try:
                            data = parse_species_page(driver, sp_url)
                        except Exception:
                            data = {
                                "accepted_name": None,
                                "species_scientific_name": None,
                                "thailand": None,
                                "distribution": None,
                                "ecology": None,
                            }

                        rows.append({
                            "volume": vol,
                            "family_name": family_name,
                            "genus_name": clean(genus_name),
                            **data,
                            "family_url": to_abs(family_url),
                            "genus_url": to_abs(genus_url),
                            "species_url": to_abs(sp_url),
                            "scraped_at": datetime.utcnow().isoformat()
                        })
                        done_species.add(sp_url)

                        # เขียนพักทุก 50 แถว
                        if len(rows) % 50 == 0:
                            pd.DataFrame(rows).to_csv(out_csv, index=False)

            # เขียนหลังจบแต่ละ volume
            pd.DataFrame(rows).to_csv(out_csv, index=False)

        # Final write + dedup
        df = pd.DataFrame(rows)
        if not df.empty:
            df = df.drop_duplicates(subset=["species_url"]).reset_index(drop=True)
        df.to_csv(out_csv, index=False)
        print(f"\n✅ เสร็จสิ้น: Saved {len(df)} rows → {out_csv}")

    finally:
        driver.quit()

# ---------- RUN ----------
if __name__ == "__main__":
    # แนะนำให้ทดสอบด้วย VOLUMES = range(2,3) ก่อน แล้วค่อยขยายเป็น range(2,17)
    crawl_all()

🧩 Resume mode: โหลด 80 แถวเดิม, species ที่ทำแล้ว 22

=== Volume 2 ===
  → พบ 46 families (หลังกรองซ้ำ)
    [1/46] Haloragaceae: 2 genera
    [2/46] Rhizophoraceae: 7 genera
    [3/46] Oxalidaceae: 3 genera
    [4/46] Ochnaceae: 4 genera


/var/folders/hb/q_224lns0mlcx0n865_2ylqc0000gn/T/ipykernel_73306/3377219641.py:290: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "scraped_at": datetime.utcnow().isoformat()


    [5/46] Rosaceae: 21 genera
    [6/46] Icacinaceae: 13 genera
    [7/46] Cardiopteridaceae: 1 genera
    [8/46] Dilleniaceae: 3 genera
    [9/46] Saurauiaceae: 1 genera
    [10/46] Schisandraceae: 1 genera
    [11/46] Illiciaceae: 1 genera
    [12/46] Connaraceae: 6 genera
    [13/46] Apostasiaceae: 2 genera
    [14/46] Actinidiaceae: 2 genera
    [15/46] Theaceae: 10 genera
    [16/46] Bonnetiaceae: 1 genera
    [17/46] Centrolepidaceae: 1 genera
    [18/46] Flagellariaceae: 1 genera
    [19/46] Hanguanaceae: 1 genera
    [20/46] Juncaceae: 1 genera
    [21/46] Lowiaceae: 1 genera
    [22/46] Restionaceae: 1 genera
    [23/46] Triuridaceae: 1 genera
    [24/46] Balanophoraceae: 1 genera
    [25/46] Rafflesiaceae: 2 genera
    [26/46] Cycadaceae: 1 genera
    [27/46] Pinaceae: 1 genera
    [28/46] Cephalotaxaceae: 1 genera
    [29/46] Cupressaceae: 1 genera
    [30/46] Podocarpaceae: 2 genera
    [31/46] Gnetaceae: 1 genera
    [32/46] Smilacaceae: 2 genera
    [33/46] Magnoliaceae: